In [3]:
library(readr)
library(GenomicFeatures)
library(DESeq2)
library(org.Mm.eg.db)
library(rjson)
library(tximport)
library(DBI)
library(rje)
library(plyr)
library(tidyverse)

In [2]:
code.dir <- getwd()
base.dir <- gsub('/codes', '', code.dir)
base.dir

[1] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/2020_mouse_CD8_additional"

In [4]:
wk.dir <- file.path(base.dir, "2_DEseq_out")
setwd(wk.dir)

In [5]:
######################################## Convert ENSEMBL ID to gene symbols ########################################
# Download convert table from:http://useast.ensembl.org/biomart/martview/8c1957c27101a044a318d51140a289e1

cv_file <- '/home/pipkin/references/mm_BioMart_GeneStableID_GeneName.txt'
cv_tb <- read_csv(cv_file)

matchGN <- function(input, outfilename, cvTb=cv_tb){
    colnames(input) <- c("ensembl_stable_ID", colnames(input)[2:length(colnames(input))])
    output <- cvTb %>% right_join(input, by="ensembl_stable_ID")
    output$ensembl_stable_ID <- NULL
    write_csv(output, outfilename)
}


── Column specification ────────────────────────────────────────────────────────
cols(
  ensembl_stable_ID = col_character(),
  gene_name = col_character()
)




In [9]:
###--- Make reference
#txdb <- makeTxDbFromGFF('/home/pipkin/references/GRCm38.99/Mus_musculus.GRCm38.99.gtf')
#saveDb(txdb, file='/home/pipkin/references/GRCm38.99/Mus_musculus.GRCm38.99')
mmRef <- '/home/pipkin/references/GRCm38.102/Mus_musculus.GRCm38.102'

###--- Convert transcript ID to gene ID
txdb <- loadDb(mmRef)
k <- keys(txdb, "GENEID")
res <- AnnotationDbi::select(txdb, k, "TXNAME", "GENEID")
tx2gene <- res[,2:1]

'select()' returned 1:many mapping between keys and columns



## 2016_SCIENCE_Mackay

In [51]:
mackey.sp.info <- file.path(base.dir, '1_raw_count_table/2016_SCIENCE_Mackay_rawCount_sample_info.csv')
mackey.sp.info.df <- read_csv(mackey.sp.info)

mackey.srr.info <- file.path(base.dir, '1_raw_count_table/2016_SCIENCE_Mackay_rawCount_SRA_Run_Table.txt')
mackey.srr.info.df <- read_csv(mackey.srr.info) %>% select(one_of(c("GEO_Accession (exp)", "source_name", "Tissue")))
colnames(mackey.srr.info.df) <- c("GEO_Accession", "source_name", "tissue")

mackey.srr.info.df <- mackey.srr.info.df %>% left_join(mackey.sp.info.df)

# Write
out.name <- file.path(base.dir, '1_raw_count_table/2016_SCIENCE_Mackay_rawCount_ref.csv')
write_csv(mackey.srr.info.df, out.name)

Parsed with column specification:
cols(
  GEO_Accession = col_character(),
  sp_name = col_character()
)

Parsed with column specification:
cols(
  .default = col_character(),
  AvgSpotLen = col_double(),
  Bases = col_double(),
  Bytes = col_double(),
  ReleaseDate = col_datetime(format = "")
)

See spec(...) for full column specifications.

Joining, by = "GEO_Accession"



In [163]:
mackey.anno <- file.path(base.dir, '1_raw_count_table/2016_SCIENCE_Mackay_rawCount_ref_annotated.csv')
mackey.anno.df <- read_csv(mackey.anno)

keys <- c()
for (i in mackey.anno.df$sp_name){
    i <- gsub("L003", "", i)
    i_vec <- unlist(strsplit(i, "_"))
    i_vec <- tail(i_vec,2)
    keys <- c(keys, paste(i_vec, collapse="_"))
}

mackey.anno.df$keys <- keys

mackey.anno.df.use <- mackey.anno.df %>% filter(tissue != "Bone marrow") %>% 
    filter(cond_abbr != "NK")
head(mackey.anno.df.use)

col.df <- mackey.anno.df.use %>% select(one_of('keys', 'cond_abbr')) %>% column_to_rownames(var="keys")
colnames(col.df) <- c("condition")

Parsed with column specification:
cols(
  GEO_Accession = col_character(),
  source_name = col_character(),
  tissue = col_character(),
  sp_name = col_character(),
  condition = col_character(),
  sample_name = col_character(),
  cond_abbr = col_character()
)



GEO_Accession,source_name,tissue,sp_name,condition,sample_name,cond_abbr,keys
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM1819913,Naïve CD8 T cells_spleen,Spleen,C1_C5N42ANXX_ATCACGAT,Naive,Naive_1,Naive,C5N42ANXX_ATCACGAT
GSM1819914,LCMV central memory CD8 T cells_spleen,Spleen,C2_C5N42ANXX_CGATGTAT,LCMV_Tcm_sp,LCMV_Tcm_sp_1,Tcm,C5N42ANXX_CGATGTAT
GSM1819915,LCMV effector memory CD8 T cells_spleen,Spleen,C3_C5N42ANXX_TTAGGCAT,LCMV_Tem_sp,LCMV_Tem_sp_1,Tem,C5N42ANXX_TTAGGCAT
GSM1819916,LCMV effector memory CD8 T cells_liver,Liver,C4_C5N42ANXX_TGACCAAT,LCMV_Tem_Liver,LCMV_Tem_Liver_1,Tem,C5N42ANXX_TGACCAAT
GSM1819917,LCMV tissue-resident memory CD8 T cells_liver,Liver,C5_C5N42ANXX_ACAGTGAT,LCMV_Trm_Liver,LCMV_Trm_Liver_1,Trm,C5N42ANXX_ACAGTGAT
GSM1819918,LCMV tissue-resident memory CD8 T cells_gut,Gut,C6_C5N42ANXX_GCCAATAT,LCMV_Trm_Gut,LCMV_Trm_Gut_1,Trm,C5N42ANXX_GCCAATAT


In [164]:
mackay.file <- file.path(base.dir, '1_raw_count_table/2016_SCIENCE_Mackay_rawCount.txt')
mackay.df <- read_tsv(mackay.file)

new_colnames <- c()
for (i in colnames(mackay.df)[3:length(colnames(mackay.df))]){
    i <- gsub("mm10", "",i)
    i <- gsub(".bam", "",i)
    i <- gsub("L003", "",i)
    i_vec <- unlist(strsplit( i, "_"))
    i_vec <- tail(i_vec,2)
    new_colnames <- c(new_colnames, paste(i_vec, collapse="_"))
}

colnames(mackay.df) <- c(c("EntrezID", "Symbol"), new_colnames)

mackay.df <- mackay.df %>% select(one_of(c("Symbol"), mackey.anno.df.use$keys)) %>%
    distinct(Symbol, .keep_all=TRUE)  %>% drop_na(Symbol) %>%
    column_to_rownames(var="Symbol")


head(mackay.df)

Parsed with column specification:
cols(
  .default = col_double(),
  Symbol = col_character()
)

See spec(...) for full column specifications.



,C5N42ANXX_ATCACGAT,C5N42ANXX_CGATGTAT,C5N42ANXX_TTAGGCAT,C5N42ANXX_TGACCAAT,C5N42ANXX_ACAGTGAT,C5N42ANXX_GCCAATAT,C5N42ANXX_TAGCTTAT,C5N42ANXX_GGCTACAT,C5N42ANXX_CTTGTAAT,C5N42ANXX_AGTCAACA,C5N42ANXX_AGTTCCGT,C5N42ANXX_ATGTCAGA,C6GUTANXX_GGCTACAT,C6GUTANXX_CTTGTAAT,C6GUTANXX_ATGTCAGA,C6GUTANXX_GTGAAACG,C6GUTANXX_AGTCAACA,C6GUTANXX_CCGTCCCG,C6GUTANXX_AGTTCCGT,C6GUTANXX_GTCCGCAC
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Xkr4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Gm19938,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Gm10568,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Rp1,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0
Sox17,0,0,4,0,0,0,0,0,1,3,0,0,0,0,5,0,0,0,0,0
Mrpl15,1068,730,640,621,541,914,875,604,683,588,641,620,34,12,26,19,42,30,33,63


In [165]:
dds <- DESeqDataSetFromMatrix(countData = mackay.df,
                              colData = col.df,
                              design= ~  condition)

summary(dds)

dds <- DESeq(dds)

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


[1] "DESeqDataSet object of length 26625 with 0 metadata columns"

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [167]:
contrasts <- list(c("Tcm", "Tem"), c("Tcm", "Trm"), c("Tem","Trm"),
                 c("Trm", "Naive"), c("Tem", "Naive"), c("Tcm", "Naive"))
for (i in c(1:6)) {
    contrast_i <- contrasts[[i]]
    outname_i <- paste(paste(contrast_i, collapse="_vs_"), ".csv", sep="")
    outname_i <- paste("2016_SCIENCE_Mackay", outname_i, sep="--")
    results <- as_tibble(results(dds, contrast = c("condition", contrast_i)), rownames='Symbol')
    write_csv(results, outname_i)
}

## 2018_IMMUNITY_Wang

In [26]:
wang.sp.info <- file.path(base.dir, '1_raw_count_table/2018_IMMUNITY_Wang_sample_description.csv')
wang.sp.info.df <- read_csv(wang.sp.info)
wang.sp.info.df.use <- wang.sp.info.df %>% 
    filter(Degradation == "No") %>% 
    filter(`WT-KO` != "KO")
wang.sp.info.df.use$Samples <- paste("174", wang.sp.info.df.use$Number, "quant", sep="_")
wang.sp.info.df.use

Parsed with column specification:
cols(
  Number = col_double(),
  Date = col_character(),
  `#` = col_double(),
  Description = col_character(),
  Degradation = col_character(),
  `WT-KO` = col_character(),
  condition = col_character()
)



Number,Date,#,Description,Degradation,WT-KO,condition,Samples
<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
11,3/28/17,1442,WT2-Day5-EEC,No,WT,D5EE,174_11_quant
12,3/28/17,1442,WT2-Day5-SLEC,No,WT,D5TE,174_12_quant
15,3/28/17,1443,WT3-Day5-EEC,No,WT,D5EE,174_15_quant
19,3/31/17,1442,WT2-Day8-EEC,No,WT,D8EE,174_19_quant
20,3/31/17,1442,WT2-Day8-SLEC,No,WT,D8TE,174_20_quant
21,3/31/17,1442,WT2-Day8-MPEC,No,WT,D8MP,174_21_quant
25,3/31/17,1443,WT3-Day8-EEC,No,WT,D8EE,174_25_quant
26,3/31/17,1443,WT3-Day8-SLEC,No,WT,D8TE,174_26_quant
27,3/31/17,1443,WT3-Day8-MPEC,No,WT,D8MP,174_27_quant


In [40]:
salmon_out_dir <- '/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/2020_mouse_CD8_additional/0_salmon_out/Exp174'

# Read files
files <- file.path(salmon_out_dir,wang.sp.info.df.use$Samples,"quant.sf")
nav.file.1 <- '/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/2020_mouse_CD8_additional/0_salmon_out/Exp36/Exp36_Naive1/quant.sf'
nav.file.2 <- '/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/2020_mouse_CD8_additional/0_salmon_out/Exp36/Exp36_Naive2/quant.sf'
files <- c( nav.file.1, nav.file.2,files)

# Names & Conditions
f.conds <- c("Naive", "Naive", wang.sp.info.df.use$condition)
f.names <- c("Naive1", "Naive2", wang.sp.info.df.use$Description)

names(files) <- f.names
txi <- tximport(files, type="salmon", tx2gene=tx2gene, ignoreTxVersion = TRUE, dropInfReps = TRUE) # Drop in freps TURE = ignore verison  # Ignore TX verison stringsplits on . 

# Build sample table
sampleTable <- data.frame(condition = factor(f.conds))
rownames(sampleTable) <- colnames(txi$counts)

#import into DESEQ2 framework
dds <- DESeqDataSetFromTximport(txi, sampleTable, ~ condition)
summary(dds)

###----- Run DEseq
dds <- DESeq(dds)

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 


transcripts missing from tx2gene: 282

summarizing abundance

summarizing counts

summarizing length

using counts and average transcript lengths from tximport



[1] "DESeqDataSet object of length 35721 with 0 metadata columns"

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [44]:
contrasts <- list(c("D5EE", "Naive"), c("D5TE", "Naive"),
                 c("D8EE", "D5EE"), c("D8TE", "D5EE"), c("D8MP", "D5EE"),
                 c("D8EE", "D5TE"), c("D8TE", "D5TE"), c("D8MP", "D5TE"))
for (i in c(1:8)) {
    contrast_i <- contrasts[[i]]
    out_name_i <- paste(paste(contrast_i, collapse="_vs_"), ".csv", sep="")
    out_name_i <- paste("2018_IMMUNITY_Wang", out_name_i, sep="--")
    out_name_i_gn <- gsub(".csv","_gn.csv",  out_name_i)
    results <- as_tibble(results(dds, contrast = c("condition", contrast_i)), rownames='Symbol')
    write_csv(results, out_name_i)
    matchGN(results, out_name_i_gn)
}